# Importing the libraries

In [120]:
!pip install spotipy

In [121]:
import os
import pandas as pd
import numpy as np
import json
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyOAuth,SpotifyClientCredentials
import yaml
import re
from tqdm import tqdm
import multiprocessing as mp
import time
import random
import datetime
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from skimage import io
from sklearn.preprocessing import OneHotEncoder

In [122]:
import os
print(os.path.abspath('spotify.yaml'))

/Users/devahutitalukder/spotify.yaml


In [123]:
stream= open("/Users/devahutitalukder/spotify.yaml")
spotify_details = yaml.safe_load(stream)
auth_manager = SpotifyClientCredentials(client_id=spotify_details['Client_id'],
                                        client_secret=spotify_details['Client_secret'])
sp = spotipy.client.Spotify(auth_manager=auth_manager)

In [124]:
import os
print(os.path.abspath('1M_unique_processed_data.csv'))

/Users/devahutitalukder/1M_unique_processed_data.csv


# Importing the dataset

In [125]:
dtypes = {'track_uri': 'object', 'artist_uri': 'object', 'album_uri': 'object', 'danceability': 'float16', 'energy': 'float16', 'key': 'float16',
               'loudness': 'float16', 'mode': 'float16', 'speechiness': 'float16', 'acousticness': 'float16', 'instrumentalness': 'float16',
               'liveness': 'float16', 'valence': 'float16', 'tempo': 'float16', 'duration_ms': 'float32', 'time_signature': 'float16',
               'Track_release_date': 'int8', 'Track_pop': 'int8', 'Artist_pop': 'int8', 'Artist_genres': 'object'}
try:
    df=pd.read_csv('/Users/devahutitalukder/1M_unique_processed_data.csv',dtype=dtypes)
except:
    print('Failed to load grow')
    df=pd.read_csv('/Users/devahutitalukder/1M_unique_processed_data.csv',dtype=dtypes)


# Test

In [126]:
df.head()

,track_uri,artist_uri,album_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,Track_release_date,Track_pop,Artist_pop,Artist_genres
0,0UaMYEvWZi0ZqiDOoHU3YI,2wIVse2owClT7go1WT98tk,6vV5UrXcfyQD1wu4Qo2I9K,0.903809,0.812988,4.0,-7.105469,0.0,0.120972,0.031097,0.006969,0.047089,0.810059,125.4375,226864.0,4.0,40,14,13,dance_pop hip_hop hip_pop neo_soul pop_rap r&b...
1,3jagJCUbdqhDSPuxP8cAqF,2wIVse2owClT7go1WT98tk,6DeU398qrJ1bLuryetSmup,0.883789,0.676758,1.0,-5.601562,1.0,0.282959,0.077820,0.000000,0.073181,0.583984,101.8750,263227.0,4.0,40,14,13,dance_pop hip_hop hip_pop neo_soul pop_rap r&b...
2,3XplJgPz8VjbDzbGwGgZdq,2wIVse2owClT7go1WT98tk,6epR3D622KWsnuHye7ApOl,0.793945,0.805176,0.0,-6.554688,1.0,0.229980,0.538086,0.122009,0.095215,0.658203,177.7500,236933.0,4.0,40,8,13,dance_pop hip_hop hip_pop neo_soul pop_rap r&b...
3,0jG92AlXau21qgCQRxGLic,2wIVse2owClT7go1WT98tk,20t54K6C80QQH7vbcpfJcP,0.622070,0.668945,9.0,-8.421875,1.0,0.329102,0.026596,0.000003,0.151978,0.569824,93.8125,252987.0,4.0,40,11,13,dance_pop hip_hop hip_pop neo_soul pop_rap r&b...
4,6zsk6uF3MxfIeHPlubKBvR,2wIVse2owClT7go1WT98tk,20t54K6C80QQH7vbcpfJcP,0.796875,0.750000,0.0,-9.367188,1.0,0.246948,0.533203,0.107971,0.094971,0.740234,177.8750,211120.0,4.0,40,14,13,dance_pop hip_hop hip_pop neo_soul pop_rap r&b...


Extract playlist tracks and artist uri

In [127]:
def get_IDs (user, playlist_id):
 track_ids = []
 artist_id = []
 playlist=sp.user_playlist (user, playlist_id)
 for item in playlist['tracks']['items']:
  track=item['track']
  track_ids.append(track['id'])
  artist=item['track']['artists']
  artist_id.append(artist[0]['id'])
 return track_ids,artist_id


track_ids,artist_id = get_IDs ('Ruby', 'spotify:playlist:37i9dQZF1DX8FwnYE6PRvL') 
print (len(track_ids))
print (len(artist_id))

100
100


getting the unique URI and repeating the extraction features and preprocessing steps for the user's playlist (input)

In [128]:
artist_id_uni=list(set(artist_id))
track_ids_uni=list(set(track_ids))

In [73]:
track_ids_uni

['4KacUpvbA3Mfo05gttTjhN',
 '3ZOEytgrvLwQaqXreDs2Jx',
 '2SiXAy7TuUkycRVbbWDEpo',
 '5G1sTBGbZT5o4PNRc75RKI',
 '4GvPlSOKfN7aXEuGW8zKUx',
 '7ygpwy2qP3NbrxVkHvUhXY',
 '7w87IxuO7BDcJ3YUqCyMTT',
 '0Puj4YlTm6xNzDDADXHMI9',
 '57JVGBtBLCfHw2muk5416J',
 '0Ti2dlF2xLjXblvdU5fCxM',
 '57Xjny5yNzAcsxnusKmAfA',
 '54bm2e3tk8cliUz3VSdCPZ',
 '12dU3vAh6AFoJkisorfoUl',
 '5ghIJDpPoe3CfHMGu71E6T',
 '3GBApU0NuzH4hKZq4NOSdA',
 '7N3PAbqfTjSEU1edb2tY8j',
 '2aoo2jlRnM3A0NyLQqMN2f',
 '0PdM2a6oIjqepoEfcJo0RO',
 '1Cwsd5xI8CajJz795oy4XF',
 '7j31rVgGX9Q2blT92VBEA0',
 '5qxChyzKLEyoPJ5qGrdurN',
 '5W8YXBz9MTIDyrpYaCg2Ky',
 '1hlveB9M6ijHZRbzZ2teyh',
 '1FkoVC85Ds3mFoK0fVqEqP',
 '3cfOd4CMv2snFaKAnMdnvK',
 '7snQQk1zcKl8gZ92AnueZW',
 '0rmGAIH9LNJewFw7nKzZnc',
 '5z6xHjCZr7a7AIcy8sPBKy',
 '3IV4swNduIRunHREK80owz',
 '37ZJ0p5Jm13JPevGcx4SkF',
 '549Go7a66CX6k523uK9kUv',
 '56KqaFSGTb7ifpt16t5Y1N',
 '6ZOBP3NvffbU4SZcrnt1k6',
 '1fJFuvU2ldmeAm5nFIHcPP',
 '3Pzh926pXggbMe2ZpXyMV7',
 '5PntSbMHC1ud6Vvl8x56qd',
 '39shmbIHICJ2Wxnk1fPSdz',
 

In [129]:
audio_features=pd.DataFrame()
for i in tqdm(range(0,len(track_ids_uni),25)):
    try:
     track_feature = sp.audio_features(track_ids_uni[i:i+25])
     track_df = pd.DataFrame(track_feature)
     audio_features=pd.concat([audio_features,track_df],axis=0)
    except Exception as e:
        print(e)
        continue

100%|█████████████████████████████████████████████| 4/4 [00:02<00:00,  1.81it/s]


In [130]:
audio_features.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.402,0.914,9,-5.215,1,0.1420,0.000579,0.000000,0.0650,0.626,196.505,audio_features,4KacUpvbA3Mfo05gttTjhN,spotify:track:4KacUpvbA3Mfo05gttTjhN,https://api.spotify.com/v1/tracks/4KacUpvbA3Mf...,https://api.spotify.com/v1/audio-analysis/4Kac...,178267,4
1,0.618,0.938,9,-3.442,1,0.0456,0.017900,0.000000,0.1670,0.875,91.455,audio_features,3ZOEytgrvLwQaqXreDs2Jx,spotify:track:3ZOEytgrvLwQaqXreDs2Jx,https://api.spotify.com/v1/tracks/3ZOEytgrvLwQ...,https://api.spotify.com/v1/audio-analysis/3ZOE...,269000,4
2,0.532,0.767,7,-5.509,1,0.0574,0.002870,0.000513,0.3900,0.755,127.361,audio_features,2SiXAy7TuUkycRVbbWDEpo,spotify:track:2SiXAy7TuUkycRVbbWDEpo,https://api.spotify.com/v1/tracks/2SiXAy7TuUky...,https://api.spotify.com/v1/audio-analysis/2SiX...,210173,4
3,0.356,0.872,2,-7.837,1,0.0680,0.004170,0.009750,0.0997,0.607,166.300,audio_features,5G1sTBGbZT5o4PNRc75RKI,spotify:track:5G1sTBGbZT5o4PNRc75RKI,https://api.spotify.com/v1/tracks/5G1sTBGbZT5o...,https://api.spotify.com/v1/audio-analysis/5G1s...,193653,4
4,0.706,0.821,9,-2.974,1,0.0617,0.001640,0.000000,0.0981,0.918,139.914,audio_features,4GvPlSOKfN7aXEuGW8zKUx,spotify:track:4GvPlSOKfN7aXEuGW8zKUx,https://api.spotify.com/v1/tracks/4GvPlSOKfN7a...,https://api.spotify.com/v1/audio-analysis/4GvP...,168280,4


In [131]:
track_=pd.DataFrame()
for i in tqdm(range(0,len(track_ids_uni),25)):
    try:
        track_features = sp.tracks(track_ids_uni[i:i+25])
        for x in range(25):
            track_pop=pd.DataFrame([track_ids_uni[i+x]],columns=['Track_uri'])
            track_pop['Track_release_date']=track_features['tracks'][x]['album']['release_date']
            track_pop['Track_pop'] = track_features['tracks'][x]["popularity"]
            track_pop['Artist_uri']=track_features['tracks'][x]['artists'][0]['id']
            track_pop['Album_uri']=track_features['tracks'][x]['album']['id']
            track_=pd.concat([track_,track_pop],axis=0)
    except Exception as e:
        print(e)
        continue

100%|█████████████████████████████████████████████| 4/4 [00:02<00:00,  1.90it/s]


In [132]:
track_.head()

,Track_uri,Track_release_date,Track_pop,Artist_uri,Album_uri
0,4KacUpvbA3Mfo05gttTjhN,2001-01-01,77,0qT79UgT5tY4yudH9VfsdT,2UCWsnmZEVg9HhnMeKTsim
0,3ZOEytgrvLwQaqXreDs2Jx,2002-07-09,85,0L8ExT028jH3ddEcZwqJJ5,6deiaArbeoqp1xPEGdEKp1
0,2SiXAy7TuUkycRVbbWDEpo,1980-07-25,84,711MCceyCBcFnzjGY4Q7Un,6mUdeDZCsExyJLMdAfDuwh
0,5G1sTBGbZT5o4PNRc75RKI,2011-12-06,80,7mnBLXK823vNxN3UWB7Gfz,5DLhV9yOvZ7IxVmljMXtNm
0,4GvPlSOKfN7aXEuGW8zKUx,2023-01-13,76,0lAWpj5szCSwM4rUMHYmrr,6kHJE5xnpA6zncKOD70bS8


In [133]:
artist_=pd.DataFrame()
for i in tqdm(range(0,len(artist_id_uni),25)):
    try:
        artist_features = sp.artists(artist_id_uni[i:i+25])
        for x in range(25):
            artist_df=pd.DataFrame([artist_id_uni[i+x]],columns=['Artist_uri'])
            artist_pop = artist_features['artists'][x]["popularity"]
            artist_genres = artist_features['artists'][x]["genres"]
            artist_df["Artist_pop"] = artist_pop
            if artist_genres: 
                artist_df["genres"] = " ".join([re.sub(' ','_',i) for i in artist_genres])
            else:
              artist_df["genres"] = "unknown"
            artist_=pd.concat([artist_,artist_df],axis=0)
    except Exception as e:
        print(e)
        continue

100%|█████████████████████████████████████████████| 3/3 [00:01<00:00,  1.93it/s]

list index out of range


In [134]:
artist_.head()

,Artist_uri,Artist_pop,genres
0,4F84IBURUo98rz4r61KF70,68,alternative_rock blues_rock detroit_rock garag...
0,7Ln80lUS6He07XvHI8qqHH,84,garage_rock modern_rock permanent_wave rock sh...
0,0L8ExT028jH3ddEcZwqJJ5,80,alternative_rock funk_metal funk_rock permanen...
0,6XyY86QOPPrYVGvF9ch6wz,82,alternative_metal nu_metal post-grunge rap_met...
0,2cnMpRsOVqtPMfq7YiFE6K,70,album_rock classic_rock hard_rock rock


In [164]:
test=pd.DataFrame(track_,columns=['Track_uri','Artist_uri','Album_uri'])

In [165]:
test.rename(columns = {'Track_uri':'track_uri','Artist_uri':'artist_uri','Album_uri':'album_uri'}, inplace = True)

In [137]:
audio_features.drop(columns=['type','uri','track_href','analysis_url'],axis=1,inplace=True)

In [166]:
test = pd.merge(test,audio_features, left_on = "track_uri", right_on= "id",how = 'outer')
test = pd.merge(test,track_, left_on = "track_uri", right_on= "Track_uri",how = 'outer')
test = pd.merge(test,artist_, left_on = "artist_uri", right_on= "Artist_uri",how = 'outer')

In [167]:
test.rename(columns = {'genres':'Artist_genres'}, inplace = True)

In [168]:
test.drop(columns=['Track_uri','Artist_uri_x','Artist_uri_y','Album_uri','id'],axis=1,inplace=True)

In [169]:
test.dropna(axis=0,inplace=True)

In [170]:
test['Track_pop'] = test['Track_pop'].apply(lambda x: int(x/5))
test['Artist_pop'] = test['Artist_pop'].apply(lambda x: int(x/5))
test['Track_release_date'] = test['Track_release_date'].apply(lambda x: x.split('-')[0])
test['Track_release_date']=test['Track_release_date'].astype('int16')
test['Track_release_date'] = test['Track_release_date'].apply(lambda x: int(x/50))

In [171]:
test[['danceability', 'energy', 'key','loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness','liveness', 'valence', 'tempo', 'time_signature']]=test[['danceability', 'energy', 'key','loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness','liveness', 'valence', 'tempo','time_signature']].astype('float16')
test[['duration_ms']]=test[['duration_ms']].astype('float32')
test[['Track_release_date', 'Track_pop', 'Artist_pop']]=test[['Track_release_date', 'Track_pop', 'Artist_pop']].astype('int8')

In [144]:
len(df)

199492

In [145]:
currentdf=len(df)

In [146]:
df=pd.concat([df,test],axis=0)

In [147]:
df.head()

,track_uri,artist_uri,album_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,duration_ms,time_signature,Track_release_date,Track_pop,Artist_pop,Artist_genres,Artist_genres
0,0UaMYEvWZi0ZqiDOoHU3YI,2wIVse2owClT7go1WT98tk,6vV5UrXcfyQD1wu4Qo2I9K,0.903809,0.812988,4.0,-7.105469,0.0,0.120972,0.031097,...,0.047089,0.810059,125.4375,226864.0,4.0,40,14,13,dance_pop hip_hop hip_pop neo_soul pop_rap r&b...,NaN
1,3jagJCUbdqhDSPuxP8cAqF,2wIVse2owClT7go1WT98tk,6DeU398qrJ1bLuryetSmup,0.883789,0.676758,1.0,-5.601562,1.0,0.282959,0.077820,...,0.073181,0.583984,101.8750,263227.0,4.0,40,14,13,dance_pop hip_hop hip_pop neo_soul pop_rap r&b...,NaN
2,3XplJgPz8VjbDzbGwGgZdq,2wIVse2owClT7go1WT98tk,6epR3D622KWsnuHye7ApOl,0.793945,0.805176,0.0,-6.554688,1.0,0.229980,0.538086,...,0.095215,0.658203,177.7500,236933.0,4.0,40,8,13,dance_pop hip_hop hip_pop neo_soul pop_rap r&b...,NaN
3,0jG92AlXau21qgCQRxGLic,2wIVse2owClT7go1WT98tk,20t54K6C80QQH7vbcpfJcP,0.622070,0.668945,9.0,-8.421875,1.0,0.329102,0.026596,...,0.151978,0.569824,93.8125,252987.0,4.0,40,11,13,dance_pop hip_hop hip_pop neo_soul pop_rap r&b...,NaN
4,6zsk6uF3MxfIeHPlubKBvR,2wIVse2owClT7go1WT98tk,20t54K6C80QQH7vbcpfJcP,0.796875,0.750000,0.0,-9.367188,1.0,0.246948,0.533203,...,0.094971,0.740234,177.8750,211120.0,4.0,40,14,13,dance_pop hip_hop hip_pop neo_soul pop_rap r&b...,NaN


In [148]:
df.drop_duplicates(subset=['track_uri'],inplace=True,keep='last') ## keep last to keep the dataset updated 

In [99]:
df.head(1)

,track_uri,artist_uri,album_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,...,tempo,duration_ms,time_signature,Track_release_date,Track_pop,Artist_pop,Artist_genres,genre|alternative_metal,genre|hard_rock,genre|rock
0,0UaMYEvWZi0ZqiDOoHU3YI,2wIVse2owClT7go1WT98tk,6vV5UrXcfyQD1wu4Qo2I9K,0.903809,0.812988,4.0,-7.105469,0.0,0.120972,0.031097,...,125.4375,226864.0,4.0,40,14,13,dance_pop hip_hop hip_pop neo_soul pop_rap r&b...,NaN,NaN,NaN


In [100]:
#df.dropna(axis=0,inplace=True)

In [101]:
df.head()

,track_uri,artist_uri,album_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,...,tempo,duration_ms,time_signature,Track_release_date,Track_pop,Artist_pop,Artist_genres,genre|alternative_metal,genre|hard_rock,genre|rock
0,0UaMYEvWZi0ZqiDOoHU3YI,2wIVse2owClT7go1WT98tk,6vV5UrXcfyQD1wu4Qo2I9K,0.903809,0.812988,4.0,-7.105469,0.0,0.120972,0.031097,...,125.4375,226864.0,4.0,40,14,13,dance_pop hip_hop hip_pop neo_soul pop_rap r&b...,NaN,NaN,NaN
1,3jagJCUbdqhDSPuxP8cAqF,2wIVse2owClT7go1WT98tk,6DeU398qrJ1bLuryetSmup,0.883789,0.676758,1.0,-5.601562,1.0,0.282959,0.077820,...,101.8750,263227.0,4.0,40,14,13,dance_pop hip_hop hip_pop neo_soul pop_rap r&b...,NaN,NaN,NaN
2,3XplJgPz8VjbDzbGwGgZdq,2wIVse2owClT7go1WT98tk,6epR3D622KWsnuHye7ApOl,0.793945,0.805176,0.0,-6.554688,1.0,0.229980,0.538086,...,177.7500,236933.0,4.0,40,8,13,dance_pop hip_hop hip_pop neo_soul pop_rap r&b...,NaN,NaN,NaN
3,0jG92AlXau21qgCQRxGLic,2wIVse2owClT7go1WT98tk,20t54K6C80QQH7vbcpfJcP,0.622070,0.668945,9.0,-8.421875,1.0,0.329102,0.026596,...,93.8125,252987.0,4.0,40,11,13,dance_pop hip_hop hip_pop neo_soul pop_rap r&b...,NaN,NaN,NaN
4,6zsk6uF3MxfIeHPlubKBvR,2wIVse2owClT7go1WT98tk,20t54K6C80QQH7vbcpfJcP,0.796875,0.750000,0.0,-9.367188,1.0,0.246948,0.533203,...,177.8750,211120.0,4.0,40,14,13,dance_pop hip_hop hip_pop neo_soul pop_rap r&b...,NaN,NaN,NaN


In [149]:
print('{} New Tracks Found'.format(len(df)-currentdf))

55 New Tracks Found


In [150]:
#saving the tracks if they weren't found in the dataset
if len(df)>currentdf:   
    df.to_csv('1M_unique_processed_data_grow.csv',index=False)
    print('{} New Found'.format(len(df)-currentdf))
    streamlit=df[df.Track_pop >0]             # dropped track with 0 popularity score to save space and ram for the final model
    streamlit.to_csv('streamlit.csv',index=False)
    del streamlit

55 New Found


In [151]:
df = df[~df['track_uri'].isin(test['track_uri'].values)]

In [172]:
test.columns

Index(['track_uri', 'artist_uri', 'album_uri', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature',
       'Track_release_date', 'Track_pop', 'Artist_pop', 'Artist_genres'],
      dtype='object')

In [173]:
test['Artist_genres']

0     alternative_metal canadian_pop_punk canadian_p...
1     alternative_metal canadian_pop_punk canadian_p...
2     alternative_rock funk_metal funk_rock permanen...
3     alternative_rock funk_metal funk_rock permanen...
4                        australian_rock hard_rock rock
                            ...                        
95                                 album_rock power_pop
96                                  permanent_wave rock
97                                     modern_rock rock
98    candy_pop new_wave_pop permanent_wave power_po...
99    album_rock classic_rock hard_rock rock souther...
Name: Artist_genres, Length: 100, dtype: object

In [174]:
test['Artist_genres'] = test['Artist_genres'].apply(lambda x: x.split(" "))

In [175]:
tfidf = TfidfVectorizer(max_features=3) #max_features=5 

In [176]:
test['Artist_genres']

0     [alternative_metal, canadian_pop_punk, canadia...
1     [alternative_metal, canadian_pop_punk, canadia...
2     [alternative_rock, funk_metal, funk_rock, perm...
3     [alternative_rock, funk_metal, funk_rock, perm...
4                    [australian_rock, hard_rock, rock]
                            ...                        
95                              [album_rock, power_pop]
96                               [permanent_wave, rock]
97                                  [modern_rock, rock]
98    [candy_pop, new_wave_pop, permanent_wave, powe...
99    [album_rock, classic_rock, hard_rock, rock, so...
Name: Artist_genres, Length: 100, dtype: object

In [177]:
tfidf_matrix = tfidf.fit_transform(test['Artist_genres'].apply(lambda x: " ".join(x)))

In [178]:
tfidf_matrix

<100x3 sparse matrix of type '<class 'numpy.float64'>'
	with 144 stored elements in Compressed Sparse Row format>

In [179]:
genre_df = pd.DataFrame(tfidf_matrix.toarray())

In [181]:
test['Artist_genres']

0     [alternative_metal, canadian_pop_punk, canadia...
1     [alternative_metal, canadian_pop_punk, canadia...
2     [alternative_rock, funk_metal, funk_rock, perm...
3     [alternative_rock, funk_metal, funk_rock, perm...
4                    [australian_rock, hard_rock, rock]
                            ...                        
95                              [album_rock, power_pop]
96                               [permanent_wave, rock]
97                                  [modern_rock, rock]
98    [candy_pop, new_wave_pop, permanent_wave, powe...
99    [album_rock, classic_rock, hard_rock, rock, so...
Name: Artist_genres, Length: 100, dtype: object

In [182]:
genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names_out()]

In [183]:
genre_df=genre_df.astype('float16')
test.drop(columns=['Artist_genres'],axis=1,inplace=True)

In [184]:
test = pd.concat([test.reset_index(drop=True), genre_df.reset_index(drop=True)],axis = 1)


In [185]:
test.isna().sum().sum()

0

In [186]:
df.columns

Index(['track_uri', 'artist_uri', 'album_uri', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature',
       'Track_release_date', 'Track_pop', 'Artist_pop',
       'Artist_genres           ', 'Artist_genres'],
      dtype='object')

In [192]:
df['Artist_genres           ']

0         dance_pop hip_hop hip_pop neo_soul pop_rap r&b...
1         dance_pop hip_hop hip_pop neo_soul pop_rap r&b...
2         dance_pop hip_hop hip_pop neo_soul pop_rap r&b...
3         dance_pop hip_hop hip_pop neo_soul pop_rap r&b...
4         dance_pop hip_hop hip_pop neo_soul pop_rap r&b...
                                ...                        
199487                                              unknown
199488                                              unknown
199489                                              unknown
199490                                          slam_poetry
199491                                 motown northern_soul
Name: Artist_genres           , Length: 199447, dtype: object

# df

In [193]:
df['Artist_genres'] = df['Artist_genres           '].apply(lambda x: x.split(" "))
tfidf_matrix = tfidf.transform(df['Artist_genres'].apply(lambda x: " ".join(x)))
genre_df = pd.DataFrame(tfidf_matrix.toarray())
genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names_out()]

In [194]:
genre_df=genre_df.astype('float16')
df.drop(columns=['Artist_genres'],axis=1,inplace=True)

In [195]:
df = pd.concat([df.reset_index(drop=True), genre_df.reset_index(drop=True)],axis = 1)

# pred

In [196]:
try:
    df.drop(columns=['genre|unknown'],axis=1,inplace=True)
    test.drop(columns=['genre|unknown'],axis=1,inplace=True)
except:
    print('genre|unknown not found')

genre|unknown not found


In [197]:
test.columns

Index(['track_uri', 'artist_uri', 'album_uri', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature',
       'Track_release_date', 'Track_pop', 'Artist_pop',
       'genre|alternative_metal', 'genre|hard_rock', 'genre|rock'],
      dtype='object')

In [198]:
df.columns

Index(['track_uri', 'artist_uri', 'album_uri', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature',
       'Track_release_date', 'Track_pop', 'Artist_pop',
       'Artist_genres           ', 'genre|alternative_metal',
       'genre|hard_rock', 'genre|rock'],
      dtype='object')

I was first using OneHotEncoder for "Track_release_date", "Track_pop", and "Artist_pop," but I found no difference in the final result other than high memory usage.

In [199]:
""" ohe = OneHotEncoder(handle_unknown='ignore')
dummies = pd.DataFrame(ohe.fit_transform(test[['Track_release_date', 'Track_pop', 'Artist_pop']]).toarray(), index=test.index,dtype=int)
column_name = ohe.get_feature_names_out(['Track_release_date', 'Track_pop', 'Artist_pop'])
dummies.columns=column_name
test = pd.concat([test.drop(['Track_release_date', 'Track_pop', 'Artist_pop'], axis=1), dummies], axis=1) """

" ohe = OneHotEncoder(handle_unknown='ignore')\ndummies = pd.DataFrame(ohe.fit_transform(test[['Track_release_date', 'Track_pop', 'Artist_pop']]).toarray(), index=test.index,dtype=int)\ncolumn_name = ohe.get_feature_names_out(['Track_release_date', 'Track_pop', 'Artist_pop'])\ndummies.columns=column_name\ntest = pd.concat([test.drop(['Track_release_date', 'Track_pop', 'Artist_pop'], axis=1), dummies], axis=1) "

In [200]:
""" ohe2 = OneHotEncoder(categories=ohe.categories_,handle_unknown='ignore')
dummies = pd.DataFrame(ohe2.fit_transform(df[['Track_release_date', 'Track_pop', 'Artist_pop']]).toarray(), index=df.index, dtype=int)
column_name = ohe2.get_feature_names_out(['Track_release_date', 'Track_pop', 'Artist_pop'])
dummies.columns=column_name
df=pd.concat([df.drop(['Track_release_date', 'Track_pop', 'Artist_pop'], axis=1), dummies], axis=1)
 """

" ohe2 = OneHotEncoder(categories=ohe.categories_,handle_unknown='ignore')\ndummies = pd.DataFrame(ohe2.fit_transform(df[['Track_release_date', 'Track_pop', 'Artist_pop']]).toarray(), index=df.index, dtype=int)\ncolumn_name = ohe2.get_feature_names_out(['Track_release_date', 'Track_pop', 'Artist_pop'])\ndummies.columns=column_name\ndf=pd.concat([df.drop(['Track_release_date', 'Track_pop', 'Artist_pop'], axis=1), dummies], axis=1)\n "

In [201]:
#df.info(memory_usage = "deep")

In [39]:
#test.loc[:,test.columns.str.startswith('genre')]=test.loc[:,test.columns.str.startswith('genre')].astype('bool')
#df.loc[:,df.columns.str.startswith('genre')]=df.loc[:,df.columns.str.startswith('genre')].astype('bool')


In [203]:
sc=MinMaxScaler()
df.iloc[:,3:19]=sc.fit_transform(df.iloc[:,3:19])
pickle.dump(sc, open('sc.sav', 'wb'))

In [204]:
test.iloc[:,3:19]=sc.transform(test.iloc[:,3:19])

In [205]:
playvec=pd.DataFrame(test.sum(axis=0)).T
playvec

,track_uri,artist_uri,album_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,...,valence,tempo,duration_ms,time_signature,Track_release_date,Track_pop,Artist_pop,genre|alternative_metal,genre|hard_rock,genre|rock
0,4KacUpvbA3Mfo05gttTjhN1HNE2PX70ztbEl6MLxrpNL3Z...,0qT79UgT5tY4yudH9VfsdT0qT79UgT5tY4yudH9VfsdT0L...,2UCWsnmZEVg9HhnMeKTsim2UCWsnmZEVg9HhnMeKTsim6d...,54.34375,83.75,525.0,-632.5,71.0,7.691406,5.804688,...,66.875,12600.0,23381848.0,400.0,3937,1563,1370,25.203125,23.875,60.78125


In [209]:
df.columns

Index(['track_uri', 'artist_uri', 'album_uri', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature',
       'Track_release_date', 'Track_pop', 'Artist_pop',
       'genre|alternative_metal', 'genre|hard_rock', 'genre|rock'],
      dtype='object')

In [208]:
df.drop(columns=['Artist_genres           '],axis=1,inplace=True)

In [210]:
df['sim']=cosine_similarity(df.drop(['track_uri', 'artist_uri', 'album_uri'], axis = 1),playvec.drop(['track_uri', 'artist_uri', 'album_uri'], axis = 1))
df['sim2']=cosine_similarity(df.iloc[:,16:-1],playvec.iloc[:,16:])
df['sim3']=cosine_similarity(df.iloc[:,19:-2],playvec.iloc[:,19:])
df = df.sort_values(['sim3','sim2','sim'],ascending = False,kind='stable')
qq=df.groupby('artist_uri').head(5).track_uri.head(50)     #to limit recmmendation by same artist
aa=sp.tracks(qq[0:50])
Fresult=pd.DataFrame()
for i in range(50):
    result=pd.DataFrame([i])
    result['track_name']=aa['tracks'][i]['name']
    result['artist_name']=aa['tracks'][i]['artists'][0]['name']
    #result['url']=aa['tracks'][i]['external_urls']['spotify']
    #result['image']=aa['tracks'][i]['album']['images'][1]['url']
    Fresult=pd.concat([Fresult,result],axis=0)
Fresult

,0,track_name,artist_name
0,0,Knights of Cydonia,Muse
0,1,Somebody Else,The 1975
0,2,Famous Last Words,My Chemical Romance
0,3,Madness,Muse
0,4,Robbers,The 1975
0,5,Read My Mind,The Killers
0,6,Time is Running Out,Muse
0,7,Trouble,Cage The Elephant
0,8,"Call It Fate, Call It Karma",The Strokes
0,9,Cold Cold Cold,Cage The Elephant


In [211]:
df['sim']=cosine_similarity(df.iloc[:,3:16],playvec.iloc[:,3:16])
df['sim2']=cosine_similarity(df.loc[:, df.columns.str.startswith('T')|df.columns.str.startswith('A')],playvec.loc[:, playvec.columns.str.startswith('T')|playvec.columns.str.startswith('A')])
df['sim3']=cosine_similarity(df.loc[:, df.columns.str.startswith('genre')],playvec.loc[:, playvec.columns.str.startswith('genre')])
df['sim4']=(df['sim']+df['sim2']+df['sim3'])/3
df = df.sort_values(['sim4'],ascending = False,kind='stable')
# genra>audio>pop
qq=df.groupby('artist_uri').head(5).track_uri.head(50)
aa=sp.tracks(qq[0:50])
Fresult=pd.DataFrame()
for i in range(50):
    result=pd.DataFrame([i])
    result['track_name']=aa['tracks'][i]['name']
    result['artist_name']=aa['tracks'][i]['artists'][0]['name']
    #result['url']=aa['tracks'][i]['external_urls']['spotify']
    #result['image']=aa['tracks'][i]['album']['images'][1]['url']
    Fresult=pd.concat([Fresult,result],axis=0)
Fresult

,0,track_name,artist_name
0,0,Weight of Love,The Black Keys
0,1,Challenger Part 1 - Flight,We Lost The Sea
0,2,Storm,Godspeed You! Black Emperor
0,3,Echoes - 2011 Remastered Version,Pink Floyd
0,4,The Lonely Now,Hammock
0,5,East Hastings,Godspeed You! Black Emperor
0,6,The Last Dive of David Shaw,We Lost The Sea
0,7,Dogs - 2011 Remaster,Pink Floyd
0,8,Through The Alter Ego Justifications,The Seven Mile Journey
0,9,A Moment of Silence,The Neighbourhood


In [212]:
Spotifyresult=pd.DataFrame()
for i in range(len(test)-1):
    if len(Spotifyresult)>=50:
        break
    ff=sp.recommendations(seed_tracks=list(test.track_uri[1+i:5+i]),limit=2)
    for z in range(2):
        result=pd.DataFrame([z+(2*i)+1])
        result['track_name']=ff['tracks'][z]['name']
        result['artist_name']=ff['tracks'][z]['artists'][0]['name']
        #result['uri']=ff['tracks'][z]['id']
        #result['url']=ff['tracks'][z]['external_urls']['spotify']
        #result['image']=ff['tracks'][z]['album']['images'][1]['url']
        Spotifyresult=pd.concat([Spotifyresult,result],axis=0)
Spotifyresult

,0,track_name,artist_name
0,1,No Reason,Sum 41
0,2,Alive,Pearl Jam
0,3,We Didn't Start the Fire,Billy Joel
0,4,Wheel in the Sky,Journey
0,5,Wheel in the Sky,Journey
0,6,Zero - Remastered 2012,The Smashing Pumpkins
0,7,Don't Wanna Fight,Alabama Shakes
0,8,Pour Some Sugar On Me - Remastered 2017,Def Leppard
0,9,New Moon Rising,Wolfmother
0,10,Looks That Kill,Mötley Crüe


sorry ur playlist must have atleast 5 tracks for this method to work